In [1]:
from google.colab import drive

In [ ]:
pip install cvxopt

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import cvxopt as copt

In [5]:
data = np.loadtxt('/content/drive/MyDrive/PRNN/Assignment_2/binary_class/multi_class_classification_data_group_5_train.txt', delimiter='\t',skiprows=1)
print(data.shape)

train_ratio,test_ratio = 0.7,0.3
np.random.shuffle(data)

num_samples = len(data)
num_train,num_test = int(train_ratio * num_samples),int(test_ratio * num_samples)


(14000, 11)


In [6]:
#Data Splitting
train_data,test_data = data[:num_train],data[num_train:]

print("Training set size:", len(train_data))
print("Test set size:", len(test_data))

X_train = train_data[:, :-1]  # Features
y_train = train_data[:, -1]   # Labels
X_test = test_data[:, :-1]  # Features
y_test = test_data[:, -1]   # Labels
float_array = np.array(y_test)
y_test = float_array.astype(int)
num_classes = 2

y_train,y_test = np.array(y_train * 2 - 1),np.array(y_test * 2 - 1)

Training set size: 9800
Test set size: 4200


Define Kernels

In [7]:
def linear_kernel(X1, X2): #No additional Hyperparameter
    return np.dot(X1, X2.T)


    # if X1.ndim != X2.ndim:
    #   X1 = np.expand_dims(X1, axis=1)
    #   X2 = np.expand_dims(X2, axis=1)
    # return np.exp(-gamma * np.linalg.norm(X1 - X2) ** 2)

In [8]:
def polynomial_kernel(X1, X2, degree=3): #Hyperparameter = degree
    return (np.dot(X1, X2.T) + 1) ** degree

In [9]:
def rbf_kernel(X1, X2, gamma=1.0): #HyperParameter = gamma
    n1 = np.shape(X1)[0]
    n2 = np.shape(X2)[0]
    K = np.zeros((n1, n2))
    for i in range(n1):
        for j in range(n2):
            K[i,j] = np.exp(-gamma * np.linalg.norm(X1[i] - X2[j])**2)
    return K

Define Optimization Function  (With Slack)

In [12]:
def optimize_dual(X, y, kernel, C):
    n_samples, n_features = X.shape

    # Compute the Gram matrix
    K = kernel(X, X)

    # Define the quadratic and linear terms of the QP problem
    P = copt.matrix(np.outer(y, y) * K)
    q = copt.matrix(-np.ones(n_samples))
    G = copt.matrix(np.vstack((-np.eye(n_samples), np.eye(n_samples))))
    h = copt.matrix(np.hstack((np.zeros(n_samples), np.ones(n_samples) * C)))
    A = copt.matrix(y.astype(float), (1, n_samples))
    b = copt.matrix(0.0)
    # Solve the QP problem
    soln = copt.solvers.qp(P, q, G, h, A, b)

    # Extract lagrange multipliers
    a = np.array(soln['x'])
    print(a )
    return a

Train Function

In [13]:
def train_svm(X_train, y_train, kernel, C):
    alpha = optimize_dual(X_train, y_train, kernel, C)

    # Compute support vectors
    sv_idx = alpha > 0
    sv_idx = sv_idx.flatten()
    print(sv_idx)
    support_vectors = X_train[sv_idx]
    support_vector_labels = y_train[sv_idx]
    alpha_sv = alpha[sv_idx]
    # Compute kernel matrix only for support vectors and training samples
    kernel_matrix = kernel(support_vectors, support_vectors)
    alpha_sv = alpha_sv.reshape(-1,)
    # Compute bias term
    product = (support_vector_labels * alpha_sv)
    decision_values = np.dot(kernel_matrix, product)
    bias = np.mean(support_vector_labels - decision_values)
    return support_vectors, support_vector_labels, alpha_sv, bias


Predict Function

In [14]:
def predict_svm(X_test, support_vectors, support_vector_labels, alpha_sv, bias, kernel):
    print(bias)
    decision_function = np.dot(kernel(X_test, support_vectors), (support_vector_labels * alpha_sv)) + bias
    print("decision_function",decision_function)
    print(np.sign(decision_function))
    return np.sign(decision_function)


Grid Search

In [16]:
def grid_search(X_train, y_train, X_test, y_test):
    best_accuracy,best_hyperparams,best_kernel = -1,None,None

    # Define hyperparameters and kernels to search
    # Cs = [0.1, 1.0, 10.0]
    Cs = [3.0]
    # kernels = [linear_kernel, polynomial_kernel, rbf_kernel]
    kernels = [rbf_kernel]
    for C in Cs:
        for kernel in kernels:
            # Train SVM model
            support_vectors, support_vector_labels, alpha_sv, bias = train_svm(X_train, y_train, kernel, C)
            print("Shape of Support Vectors ",support_vectors.shape)

            # Predict using trained model
            y_pred = predict_svm(X_test, support_vectors, support_vector_labels, alpha_sv, bias, kernel)

            # Evaluate accuracy
            accuracy = np.mean(y_pred == y_test)

            # Check if this model is the best so far
            if accuracy > best_accuracy:
                best_accuracy = accuracy
                best_hyperparams = {'C': C}
                if kernel == linear_kernel:
                    best_kernel = 'linear'
                elif kernel == polynomial_kernel:
                    best_kernel = 'polynomial'
                else:
                    best_kernel = 'rbf'

    return {'hyperparameters': best_hyperparams, 'kernel': best_kernel, 'accuracy': best_accuracy}

In [17]:
print("Shape of X_test:", X_test.shape)
# print("Shape of support_vectors:", support_vectors.shape)

Shape of X_test: (4200, 10)


Perform Grid Search

In [18]:
best_model = grid_search(X_train, y_train, X_test, y_test)

# Print best hyperparameters and performance metrics
print("Best Hyperparameters:", best_model['hyperparameters'])
print("Best Kernel:", best_model['kernel'])
print("Best Accuracy:", best_model['accuracy'])

     pcost       dcost       gap    pres   dres
 0: -3.4443e+03 -8.9639e+04  2e+05  8e-01  7e-15
 1: -3.2089e+03 -3.1937e+04  3e+04  2e-02  4e-15
 2: -4.4610e+03 -1.1951e+04  8e+03  4e-03  4e-15
 3: -4.9717e+03 -6.6659e+03  2e+03  3e-04  4e-15
 4: -5.1097e+03 -5.5569e+03  4e+02  2e-13  5e-15
 5: -5.1445e+03 -5.2315e+03  9e+01  1e-13  5e-15
 6: -5.1532e+03 -5.1618e+03  9e+00  1e-13  5e-15
 7: -5.1542e+03 -5.1547e+03  4e-01  4e-14  5e-15
 8: -5.1543e+03 -5.1543e+03  2e-02  2e-13  5e-15
 9: -5.1543e+03 -5.1543e+03  4e-04  1e-13  5e-15
Optimal solution found.
[[6.76157564e-07]
 [3.41338519e-01]
 [9.63748487e-01]
 ...
 [7.48876192e-01]
 [2.63901519e-08]
 [2.99999997e+00]]
[ True  True  True ...  True  True  True]
Shape of Support Vectors  (9800, 10)
-0.29283064893969446
decision_function [-0.43767079  1.56442211 -0.82879573 ... -0.79193835 -0.52773403
  0.14680877]
[-1.  1. -1. ... -1. -1.  1.]
Best Hyperparameters: {'C': 3.0}
Best Kernel: rbf
Best Accuracy: 0.7480952380952381
